In [1]:
import os
os.environ['KAGGLE_CONFIG_DIR'] ='/content'
!kaggle datasets download -d yasserh/breast-cancer-dataset
!unzip \*.zip && rm *.zip

  0% 0.00/48.6k [00:00<?, ?B/s]
100% 48.6k/48.6k [00:00<00:00, 30.3MB/s]
Archive:  breast-cancer-dataset.zip
  inflating: breast-cancer.csv       


In [4]:
!pip install scikit-fuzzy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 994.0/994.0 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-fuzzy: filename=scikit_fuzzy-0.4.2-py3-none-any.whl size=894078 sha256=f7bc94f10fe535ece41e19a66bfd89e0d1fca0bc127391de3b8cb4a16ff991ca
  Stored in directory: /root/.cache/pip/wheels/4f/86/1b/dfd97134a2c8313e519bcebd95d3fedc7be7944db022094bc8
Successfully built scikit-fuzzy


In [7]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import skfuzzy as fuzz

# Step 1: Load Data
data = pd.read_csv('/content/breast-cancer.csv')

# Assuming 'diagnosis' is the target variable
X = data[['radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean',
          'smoothness_mean', 'compactness_mean', 'concavity_mean', 'concave points_mean']]
y = data['diagnosis']  # Target variable

# Step 2: Splitting the dataset into the Training set and Test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: Feature Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Step 4: Model Training - FKNN
# Define a fuzzy membership function
def membership_function(x):
    return fuzz.gaussmf(x, 0, 1)  # Example Gaussian membership function

# Compute membership values for each data point in the training set
membership_values = np.array([membership_function(x) for x in X_train_scaled])

# Compute fuzzy weights for each neighbor
fuzzy_weights = membership_values / np.sum(membership_values, axis=0)

from sklearn.preprocessing import LabelEncoder

# Initialize label encoder
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)

# Initialize list to store predictions
y_pred = []

# Classify each test data point using FKNN
for x_test_point in X_test_scaled:
    # Compute distances to training data points
    distances = np.linalg.norm(X_train_scaled - x_test_point, axis=1)
    # Compute weighted votes for each class
    class_votes = np.zeros(2)  # Assuming binary classification (Malignant and Benign)
    for i, weight in enumerate(fuzzy_weights):
        nearest_neighbor_class = y_train_encoded[i]
        class_votes[nearest_neighbor_class] += weight * distances[i]  # Weighted by distance and membership
    # Choose the class with the highest total vote
    predicted_class = np.argmax(class_votes)
    y_pred.append(predicted_class)



# Step 5: Model Evaluation
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

print("Classification Report:\n", classification_report(y_test, y_pred))


ValueError: setting an array element with a sequence.